In [1]:
import torch
from datasets import load_dataset
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    RobertaTokenizer, RobertaForSequenceClassification,
    XLNetTokenizer, XLNetForSequenceClassification,
    TrainingArguments, Trainer
)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import numpy as np


C:\Users\halpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check if GPU is available and set the device accordingly
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Initialize tokenizers and models
models_and_tokenizers = {
    "bert": (BertTokenizer.from_pretrained('bert-base-uncased'), BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)),
    "roberta": (RobertaTokenizer.from_pretrained('roberta-base'), RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)),
    "xlnet": (XLNetTokenizer.from_pretrained('xlnet-base-cased'), XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=2)),
}

# Define the tokenization function
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

# Apply the tokenization function to the dataset
tokenized_datasets = {name: dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True) for name, (tokenizer, _) in models_and_tokenizers.items()}

# Split the dataset into train and validation sets
train_test_splits = {name: tokenized_datasets[name]['train'].train_test_split(test_size=0.2) for name in models_and_tokenizers}
train_datasets = {name: split['train'].shuffle(seed=42) for name, split in train_test_splits.items()}  # Shuffle the train dataset
valid_datasets = {name: split['test'].shuffle() for name, split in train_test_splits.items()}

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define training arguments for each model
training_args_dict = {
    "bert": TrainingArguments(
        output_dir='./bert_results',
        evaluation_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
    ),
    "roberta": TrainingArguments(
        output_dir='./roberta_results',
        evaluation_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
    ),
    "xlnet": TrainingArguments(
        output_dir='./xlnet_results',
        evaluation_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
    ),
}



def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\halpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages dire

In [3]:
# Train, evaluate, and save each model
accuracies = {}
for name, (tokenizer, model) in models_and_tokenizers.items():
    print(f"Training and evaluating {name} model...")
    model.to(device)
    
    # Define Trainer with model, arguments, and datasets
    trainer = Trainer(
        model=model,
        args=training_args_dict[name],
        train_dataset=train_datasets[name],
        eval_dataset=valid_datasets[name],
        compute_metrics=compute_metrics
    )

    # Start training
    trainer.train()

    # Evaluate the model
    metrics = trainer.evaluate()
    accuracies[name] = metrics['eval_accuracy']
    print(f"Metrics for {name} model:", metrics)
    
    # Save the model weights
    model.save_pretrained(f'./{name}_model')

print("Training and saving of all models completed.")
print("Accuracies of each model:")
for name, accuracy in accuracies.items():
    print(f"{name} model accuracy: **{accuracy:.4f}**")

Training and evaluating bert model...


  7%|▋         | 500/7500 [03:32<50:15,  2.32it/s]

{'loss': 0.3716, 'grad_norm': 19.597900390625, 'learning_rate': 9.333333333333334e-06, 'epoch': 0.2}


 13%|█▎        | 1000/7500 [07:05<45:36,  2.38it/s] 

{'loss': 0.313, 'grad_norm': 0.5496721267700195, 'learning_rate': 8.666666666666668e-06, 'epoch': 0.4}


...

{'loss': 0.1145, 'grad_norm': 0.04412345588207245, 'learning_rate': 0.0, 'epoch': 3.0}



100%|██████████| 7500/7500 [1:55:55<00:00,  1.08it/s]


{'eval_loss': 0.32974615693092346, 'eval_accuracy': 0.9418, 'eval_f1': 0.9418021170158271, 'eval_precision': 0.9418103386682595, 'eval_recall': 0.9418, 'eval_runtime': 217.8816, 'eval_samples_per_second': 22.948, 'eval_steps_per_second': 2.869, 'epoch': 3.0}
{'train_runtime': 6955.6355, 'train_samples_per_second': 8.626, 'train_steps_per_second': 1.078, 'train_loss': 0.1870669687906901, 'epoch': 3.0}


100%|██████████| 625/625 [03:37<00:00,  2.88it/s]


Metrics for xlnet model: {'eval_loss': 0.32974615693092346, 'eval_accuracy': 0.9418, 'eval_f1': 0.9418021170158271, 'eval_precision': 0.9418103386682595, 'eval_recall': 0.9418, 'eval_runtime': 217.5542, 'eval_samples_per_second': 22.983, 'eval_steps_per_second': 2.873, 'epoch': 3.0}
Training and saving of all models completed.
Accuracies of each model:
bert model accuracy: **0.9298**
roberta model accuracy: **0.9456**
xlnet model accuracy: **0.9418**
